In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import helpers

In [3]:
"""
Version 1: TIMEOUT
- used methods: Iterative DP

"""


# Solution for GENIUS
def solution(songs, prob, favorites, time):
    n = len(songs)

    # Init cache
    cache = [[0.0] * n for _ in range(5)]
    cache[0][0] = 1.0

    # Calculate all
    for t in range(1, time + 1):
        for next in range(n):
            p = 0
            for prev in range(n):
                p += prob[prev][next] * cache[(t - songs[prev]) % 5][prev]

            cache[t % 5][next] = p

    # Collect answers
    result = []
    for q in favorites:
        tmp = sum(cache[k % 5][q]
                  for k in range(time - songs[q] + 1, time + 1))
        result.append(tmp)

    return result


# Main I/O part
def main(rl):
    C = int(rl())
    for _ in range(C):
        N, K, M = map(int, rl().split())
        L = list(map(int, rl().split()))
        T = []
        for _ in range(N):
            row = list(map(float, rl().split()))
            T.append(row)

        Q = list(map(int, rl().split()))

        result = solution(L, T, Q, K)
        print(*map(lambda f: format(f, ".8f"), result))


# Additional codes to simulate I/O
try:
    import IPython
except ImportError as _:
    # Submit env
    import sys
    main(sys.stdin.readline)
else:
    from helpers import runner
    # IPython env
    runner.run(main)

In [4]:
"""
Version 2: TIMEOUT
- used methods: Matrix

"""


def matmul(a, b):
    l, m, n, p = len(a), len(a[0]), len(b), len(b[0])
    assert m == n

    c = [[0.0] * p for _ in range(l)]
    for i in range(l):
        for j in range(p):
            tmp = 0
            for k in range(m):
                tmp += a[i][k] * b[k][j]
            c[i][j] = tmp
    return c


def matpow(a, k):
    if k == 1:
        return a

    if k % 2 == 1:
        return matmul(a, matpow(a, k - 1))

    temp = matpow(a, k // 2)
    return matmul(temp, temp)


# Solution for GENIUS
def solution(songs, prob, favorites, time):
    n = len(songs)

    # Init W
    W = [[0.0] * (4 * n) for _ in range(4 * n)]
    for i in range(3 * n):
        W[i][i + n] = 1.0

    for i in range(n):
        for j in range(n):
            W[3 * n + i][(4 - songs[j]) * n + j] = prob[j][i]

    W = matpow(W, time)

    # Collect answers
    result = []
    for q in favorites:
        tmp = 0
        for s in range(songs[q]):
            tmp += W[(3 - s) * n + q][3 * n]

        result.append(tmp)

    return result


# Main I/O part
def main(rl):
    C = int(rl())
    for _ in range(C):
        N, K, M = map(int, rl().split())
        L = list(map(int, rl().split()))
        T = []
        for _ in range(N):
            row = list(map(float, rl().split()))
            T.append(row)

        Q = list(map(int, rl().split()))

        result = solution(L, T, Q, K)
        print(*map(lambda f: format(f, ".8f"), result))


# Additional codes to simulate I/O
try:
    import IPython
except ImportError as _:
    # Submit env
    import sys
    main(sys.stdin.readline)
else:
    from helpers import runner
    # IPython env
    runner.run(main)

In [5]:
"""
Testing

"""
from itertools import zip_longest
from helpers import testing

inf = float('inf')


def element_epsilon(a_list, b_list):
    return all(testing.epsilon(a, b)
               for a, b in zip_longest(a_list, b_list, fillvalue=inf))


test_cases = [
    {
        'input': [
            [4, 4, 2],
            [[0.18, 0.40, 0.42],
             [0.15, 0.46, 0.39],
             [0.58, 0.23, 0.19]],
            [0, 1, 2],
            6,
        ],
        'expected': [0.42360000, 0.49660000, 0.07980000],
        'oracle': element_epsilon,
    },
    {
        'input': [
            [1, 3, 2, 4],
            [[0.26, 0.07, 0.49, 0.18],
             [0.21, 0.33, 0.15, 0.31],
             [0.41, 0.20, 0.38, 0.01],
             [0.28, 0.31, 0.18, 0.23]],
            [2, 0, 3, 1],
            10,
        ],
        'expected': [0.31060929, 0.13791635, 0.26756048, 0.28391388],
        'oracle': element_epsilon,
    },
    {
        'input': [
            [4, 3, 4, 4],
            [[0.08, 0.47, 0.12, 0.33],
             [0.10, 0.02, 0.39, 0.49],
             [0.08, 0.33, 0.35, 0.24],
             [0.14, 0.19, 0.58, 0.09]],
            [1, 3, 2, 0],
            1000,
        ],
        'expected': [0.18648004, 0.28409359, 0.42243515, 0.10699122],
        'oracle': element_epsilon,
        'verbose': True,
    },
    {
        'input': [
            [5] * 50,
            [[0.02] * 50 for _ in range(50)],
            list(range(10)),
            1000000,
        ],
        'expected': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        'oracle': element_epsilon,
        'timeout': 120,
        'verbose': True,
    }
]

testing.run(solution, test_cases)

[0.00118490] solution([4, 4, 2], [[0.18, 0.4, 0.42], [0.15, 0.) → [0.4236, 0.49660000000000004, 0.0798]
Test 1 passed

[0.00352590] solution([1, 3, 2, 4], [[0.26, 0.07, 0.49, 0.18],) → [0.3106092898526644, 0.1379163489661948, 0.26756047955360357, 0.2839138816275373
Test 2 passed

[0.02036220] solution([4, 3, 4, 4], [[0.08, 0.47, 0.12, 0.33],) → [0.1864800415874776, 0.2840935884930199, 0.42243515239192225, 0.10699121752758]
Test 3 passed


             163 function calls (149 primitive calls) in 0.021 seconds

       Ordered by: cumulative time

       ncalls  tottime  percall  cumtime  percall filename:lineno(function)
            1    0.000    0.000    0.021    0.021 /workspace/helpers/src/decorators.py:120(wrapper)
            1    0.000    0.000    0.021    0.021 /workspace/helpers/src/decorators.py:58(wrapper)
            1    0.000    0.000    0.020    0.020 <ipython-input-4-6bb768ce15a0>:34(solution)
         15/1    0.000    0.000    0.020    0.020 <ipython-input-4-6bb768ce15a0>